# 04 · Blend & Eval — ансамбль ранкеров, стабильность и финальный инференс 🧪

Цель: объединить 2–6 ранкеров в устойчивый бленд (RRF/Borda/Comb*, весовой, level-2), проверить метрики на OOF,
робастность по срезам и стабильность бутстрэпом, и выдать финальный `ranked.parquet` для сабмита.

Артефакты:
`artifacts/recsys/ensemble/<dataset>/<profile>/<ens_id>/{ranked.parquet, scores_blend.parquet, weights.json, blender_config.json, oof_metrics.json, slices.json, stability.json, report.json}`

In [ ]:
from IPython.display import display, HTML
display(HTML("""
<style>
/* ipywidgets v8 (JupyterLab 4) */
.jp-OutputArea .widget-button .widget-label { 
  white-space: normal !important; 
  overflow: visible !important; 
  text-overflow: clip !important;
  line-height: 1.2 !important;
}
/* fallback для ipywidgets v7 */
.jupyter-widgets.widget-button .widget-label {
  white-space: normal !important; 
  overflow: visible !important; 
  text-overflow: clip !important;
  line-height: 1.2 !important;
}
</style>
"""))


In [ ]:
import os, sys, json, time, math, warnings, random
from dataclasses import dataclass
from datetime import datetime
from pathlib import Path
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd

from IPython.display import display, clear_output, HTML
import ipywidgets as W

warnings.filterwarnings("ignore")
pd.set_option("display.max_colwidth", 140)

# ML utils
try:
    from sklearn.isotonic import IsotonicRegression
    from sklearn.linear_model import LogisticRegression, Ridge
    from sklearn.model_selection import KFold
except Exception:
    IsotonicRegression = None
    LogisticRegression = None
    Ridge = None
    KFold = None

REPO = Path.cwd()

def mem_gb(obj=None):
    try:
        import psutil
        if obj is None: return psutil.Process().memory_info().rss/(1024**3)
    except Exception:
        pass
    try:
        if hasattr(obj, "memory_usage"):
            return float(obj.memory_usage(deep=True).sum())/(1024**3)
        return float(np.array(obj).nbytes)/(1024**3)
    except Exception:
        return float("nan")

def save_json(path: Path, obj: dict):
    path.parent.mkdir(parents=True, exist_ok=True)
    path.write_text(json.dumps(obj, ensure_ascii=False, indent=2))

def now_tag(): 
    return datetime.now().strftime("%m%d_%H%M")

def art_paths(dataset_id, profile_path):
    prof = Path(profile_path).stem
    root = Path("artifacts")
    return dict(
        dataio  = root/"recsys"/"dataio"/dataset_id,
        rankers = root/"recsys"/"ranker"/dataset_id/prof,
        ens     = root/"recsys"/"ensemble"/dataset_id/prof,
        profile = root/"recsys"/"dataio"/dataset_id/prof
    )

def exists(p): 
    try: return Path(p).exists()
    except: return False

def list_ranker_runs(dataset_id, profile_path) -> List[Path]:
    A = art_paths(dataset_id, profile_path)
    d = A["rankers"]
    if not d.exists(): return []
    runs = [p for p in sorted(d.iterdir()) if p.is_dir()]
    # фильтруем по наличию oof или ranked
    out = []
    for r in runs:
        if (r/"oof.parquet").exists() or (r/"ranked.parquet").exists():
            out.append(r)
    return out

def read_parquet_safe(path: Path, cols: List[str] = None):
    df = pd.read_parquet(path)
    if cols:
        miss = [c for c in cols if c not in df.columns]
        if miss: 
            raise AssertionError(f"{path} не содержит столбцы: {miss}")
        return df[cols].copy()
    return df

def safe_rank_in_group(df, by=["query_id","score"], ascending=[True,False], k_col="rank"):
    df = df.sort_values(by, ascending=ascending)
    df[k_col] = df.groupby("query_id")["score"].rank(ascending=False, method="first").astype("int32")
    return df

In [ ]:
BTN_LAYOUT = W.Layout(min_width="220px", width="auto", height="36px", flex="0 0 auto")
ROW_LAYOUT = W.Layout(flex_flow="row wrap", grid_gap="8px")
GRID_LAYOUT = W.Layout(grid_template_columns="repeat(3, minmax(220px, 1fr))", grid_gap="8px")

In [ ]:
STATE = dict(
    DATASET_ID   = "s5e11",
    PROFILE_PATH = "configs/recsys/profiles/gate.yaml",
    K_OUT        = 100,
    RANDOM_SEED  = 42,
)

EVAL = dict(
    KS = [10,20,50,100],
    MAIN_K = 20
)

NORM = dict(
    method = "zscore_query",  # zscore_query | minmax_query | quantile_query | none
    calibrate = "none"        # none | platt | isotonic
)

BLEND = dict(
    use_rrf=True, rrf_k=60,
    use_borda=True,
    use_combsum=True, use_combmnz=True,
    use_weight_search=True, weight_nonneg=True, weight_sum_to_one=True, weight_trials=300,
    use_meta_l2=True, meta_algo="ridge", meta_splits=5
)

# Виджеты
w_ds   = W.Text(STATE["DATASET_ID"], description="dataset_id:", layout=W.Layout(width="240px"))
w_prof = W.Text(STATE["PROFILE_PATH"], description="profile:", layout=W.Layout(width="520px"))
w_kout = W.IntText(STATE["K_OUT"], description="K_OUT:", layout=W.Layout(width="160px"))
w_seed = W.IntText(STATE["RANDOM_SEED"], description="seed:", layout=W.Layout(width="160px"))

w_ks   = W.Text(",".join(map(str,EVAL["KS"])), description="KS:", layout=W.Layout(width="220px"))
w_main = W.IntText(EVAL["MAIN_K"], description="main@K:", layout=W.Layout(width="180px"))

w_norm = W.Dropdown(options=["zscore_query","minmax_query","quantile_query","none"], value=NORM["method"], description="normalize:")
w_cal  = W.Dropdown(options=["none","platt","isotonic"], value=NORM["calibrate"], description="calibrate:")

tog_rrf   = W.Checkbox(BLEND["use_rrf"], description="RRF")
w_rrf_k   = W.IntText(BLEND["rrf_k"], description="rrf_k:")
tog_borda = W.Checkbox(BLEND["use_borda"], description="Borda")
tog_csum  = W.Checkbox(BLEND["use_combsum"], description="CombSUM")
tog_cmnz  = W.Checkbox(BLEND["use_combmnz"], description="CombMNZ")
tog_wght  = W.Checkbox(BLEND["use_weight_search"], description="WeightSearch")
w_trials  = W.IntText(BLEND["weight_trials"], description="trials:")
w_nonneg  = W.Checkbox(BLEND["weight_nonneg"], description="nonneg")
w_sum1    = W.Checkbox(BLEND["weight_sum_to_one"], description="sum_to_one")
tog_meta  = W.Checkbox(BLEND["use_meta_l2"], description="Level-2 (Ridge)")
w_malgo   = W.Dropdown(options=["ridge"], value=BLEND["meta_algo"], description="meta:")
w_msplits = W.IntText(BLEND["meta_splits"], description="splits:")

btn_scan  = W.Button(description="Сканировать ранкеры", button_style="primary", layout=BTN_LAYOUT)
out_scan  = W.Output()

# Placeholder для списка ранкеров
SEL = dict(runs=[], widget=None)

def on_scan(_):
    with out_scan:
        clear_output()
        # apply
        STATE.update(DATASET_ID=w_ds.value.strip(), PROFILE_PATH=w_prof.value.strip(), K_OUT=int(w_kout.value), RANDOM_SEED=int(w_seed.value))
        EVAL.update(KS=[int(x) for x in w_ks.value.split(",") if x.strip()], MAIN_K=int(w_main.value))
        NORM.update(method=w_norm.value, calibrate=w_cal.value)
        BLEND.update(
            use_rrf=bool(tog_rrf.value), rrf_k=int(w_rrf_k.value),
            use_borda=bool(tog_borda.value),
            use_combsum=bool(tog_csum.value), use_combmnz=bool(tog_cmnz.value),
            use_weight_search=bool(tog_wght.value), weight_trials=int(w_trials.value),
            weight_nonneg=bool(w_nonneg.value), weight_sum_to_one=bool(w_sum1.value),
            use_meta_l2=bool(tog_meta.value), meta_algo=w_malgo.value, meta_splits=int(w_msplits.value)
        )

        runs = list_ranker_runs(STATE["DATASET_ID"], STATE["PROFILE_PATH"])
        if not runs:
            print("Не найдено ранкеров. Сначала прогони 03_ranker_lab.")
            return
        SEL["runs"]=runs
        options = [(f"{r.name}  —  {r}", str(r)) for r in runs]
        SEL["widget"] = W.SelectMultiple(options=options, description="runs", rows=min(10, len(options)))
        display(HTML("<b>Выбери 2–6 ранкеров для бленда</b>"))
        display(SEL["widget"])

btn_scan.on_click(on_scan)

W.VBox([
    W.HTML("<h3>Параметры</h3>"),
    W.HBox([w_ds, w_seed, w_kout], layout=ROW_LAYOUT),
    w_prof,
    W.HTML("<b>Метрики</b>"),
    W.HBox([w_ks, w_main], layout=ROW_LAYOUT),
    W.HTML("<b>Нормализация/калибровка</b>"),
    W.HBox([w_norm, w_cal], layout=ROW_LAYOUT),
    W.HTML("<b>Блендинг</b>"),
    W.HBox([tog_rrf, w_rrf_k, tog_borda, tog_csum, tog_cmnz], layout=ROW_LAYOUT),
    W.HBox([tog_wght, w_trials, w_nonneg, w_sum1], layout=ROW_LAYOUT),
    W.HBox([tog_meta, w_malgo, w_msplits], layout=ROW_LAYOUT),
    btn_scan, out_scan
])

In [ ]:
btn_load = W.Button(description="Загрузить OOF/TEST выбранных ранкеров", button_style="success", layout=BTN_LAYOUT)
out_load = W.Output()

RUNS = dict()  # run_id -> {path, oof_df, test_df, K_in}

def _load_single_run(run_path: Path):
    oof_path  = run_path/"oof.parquet"
    test_path = run_path/"test_scores.parquet"
    ranked_tk = run_path/"ranked.parquet"
    if not exists(oof_path):
        raise AssertionError(f"Нет {oof_path}")
    oof = read_parquet_safe(oof_path, ["query_id","item_id","label","oof_score"]).rename(columns={"oof_score":"score"})
    # test
    if exists(test_path):
        test = read_parquet_safe(test_path, ["query_id","item_id","score"])
    elif exists(ranked_tk):
        test = read_parquet_safe(ranked_tk, ["query_id","item_id","score"])
    else:
        raise AssertionError(f"Нет test_scores/ranked в {run_path}")
    # K_in — среднее кол-во кандидатов на query в OOF
    K_in = int(oof.groupby("query_id").size().mean())
    return oof, test, K_in

def on_load(_):
    with out_load:
        clear_output()
        chosen = SEL.get("widget")
        if not chosen:
            print("Сначала отсканируй ранкеры и выбери их.")
            return
        sel_ids = [Path(v) for v in chosen.value]
        if len(sel_ids)<2:
            print("Нужно ≥2 ранкеров.")
            return
        random.seed(STATE["RANDOM_SEED"]); np.random.seed(STATE["RANDOM_SEED"])
        RUNS.clear()
        for rp in sel_ids:
            oof, test, kin = _load_single_run(rp)
            rid = rp.name
            RUNS[rid] = dict(path=rp, oof=oof, test=test, K_in=kin)
            print(f"OK {rid}: OOF {oof.shape} | TEST {test.shape} | ~K_in≈{kin}")
        # подсводка и корреляции (по пересечению)
        ids = list(RUNS.keys())
        print("
Пересечения OOF по парам ранкеров (кол-во общих (q,i)):")
        M = []
        for i,a in enumerate(ids):
            row=[]
            for j,b in enumerate(ids):
                if j<i: row.append("")
                else:
                    aa = RUNS[a]["oof"][ ["query_id","item_id"] ]
                    bb = RUNS[b]["oof"][ ["query_id","item_id"] ]
                    inter = aa.merge(bb, on=["query_id","item_id"], how="inner").shape[0]
                    row.append(inter)
            M.append(row)
        display(pd.DataFrame(M, index=ids, columns=ids))

btn_load.on_click(on_load)
W.VBox([btn_load, out_load])

In [ ]:
btn_norm = W.Button(description="Нормализовать/калибровать OOF", button_style="warning", layout=BTN_LAYOUT)
out_norm = W.Output()

NORMED = dict()  # run_id -> {oof_norm, test_norm, per_query_norm? — не нужен; калибратор}

def _normalize_querywise(df: pd.DataFrame, method: str):
    # df: (query_id, item_id, score)
    d = df.copy()
    if method=="none":
        return d
    if method=="zscore_query":
        g = d.groupby("query_id")["score"]
        mu = g.transform("mean"); sd = g.transform("std").replace(0, 1.0)
        d["score"] = (d["score"] - mu) / sd
    elif method=="minmax_query":
        g = d.groupby("query_id")["score"]
        mn = g.transform("min"); mx = g.transform("max")
        d["score"] = (d["score"] - mn) / (mx - mn + 1e-9)
    elif method=="quantile_query":
        # заменим ранги на равномерные квантильки (1..K)/K
        d = d.sort_values(["query_id","score"], ascending=[True,False])
        d["score"] = d.groupby("query_id")["score"].rank(ascending=False, method="first")
        d["score"] = d["score"] / d.groupby("query_id")["score"].transform("max")
    else:
        return d
    return d

def _fit_calibrator(scores, labels, method="none"):
    if method=="none":
        return None
    if method=="platt":
        if LogisticRegression is None:
            print("sklearn не установлен — Platt недоступен"); return None
        lr = LogisticRegression(max_iter=1000, n_jobs=None if hasattr(LogisticRegression, "n_jobs") else None)
        lr.fit(scores.reshape(-1,1), labels.astype(int))
        return ("platt", lr)
    if method=="isotonic":
        if IsotonicRegression is None:
            print("sklearn не установлен — Isotonic недоступен"); return None
        ir = IsotonicRegression(out_of_bounds="clip")
        ir.fit(scores, labels.astype(float))
        return ("isotonic", ir)
    return None

def _apply_calibrator(scores, calib):
    if calib is None:
        return scores
    kind, model = calib
    if kind=="platt":
        p = model.predict_proba(scores.reshape(-1,1))[:,1]
        return p.astype("float32")
    if kind=="isotonic":
        p = model.predict(scores)
        return p.astype("float32")
    return scores

def on_norm(_):
    with out_norm:
        clear_output()
        if not RUNS:
            print("Сначала загрузите ранкеры.")
            return
        NORMED.clear()
        for rid, R in RUNS.items():
            oof = R["oof"][ ["query_id","item_id","label","score"] ].copy()
            test= R["test"][ ["query_id","item_id","score"] ].copy()
            # normalize per query
            oofn = _normalize_querywise(oof.drop(columns=["label"]), NORM["method"])
            testn= _normalize_querywise(test, NORM["method"])
            # calibrate on OOF if requested
            calib = None
            if NORM["calibrate"]!="none":
                calib = _fit_calibrator(oofn["score"].values, oof["label"].values, NORM["calibrate"])
                if calib is not None:
                    oofn["score"]  = _apply_calibrator(oofn["score"].values, calib)
                    testn["score"] = _apply_calibrator(testn["score"].values, calib)
            # cache
            NORMED[rid] = dict(
                oof=oofn, oof_label=oof["label"].values.copy(),
                test=testn, calib=None if calib is None else NORM["calibrate"]
            )
            print(f"✓ {rid} нормализован ({NORM['method']}, calib={NORM['calibrate']})")
        print("Готово.")

btn_norm.on_click(on_norm)
W.VBox([btn_norm, out_norm])

In [ ]:
def recall_ndcg_at_k(qids, labels, scores, ks=(10,20,50,100)):
    df = pd.DataFrame({"q":qids, "y":labels, "s":scores})
    df = df.sort_values(["q","s"], ascending=[True,False])
    out={}
    for K in ks:
        hits=0; tot=0; dcg=0.0; idcg=0.0
        for q, grp in df.groupby("q", sort=False):
            arr = grp["y"].values[:K]
            hit = int(arr.sum()>0)
            hits += hit; tot += 1
            nd=0.0
            for r,v in enumerate(arr, start=1):
                if v>0:
                    nd = 1.0/math.log2(r+1); break
            dcg += nd; idcg += 1.0
        out[f"recall@{K}"]=hits/max(1,tot)
        out[f"ndcg@{K}"]=dcg/max(1.0,idcg)
    return out

def union_queries_items(runs_oof: Dict[str, pd.DataFrame]) -> pd.DataFrame:
    # Собираем union по (query_id,item_id), присоединяем score_<rid> и label
    keys = list(runs_oof.keys())
    # старт: первый
    rid0 = keys[0]
    base = runs_oof[rid0][["query_id","item_id"]].drop_duplicates().copy()
    for rid in keys[1:]:
        base = base.merge(runs_oof[rid][["query_id","item_id"]].drop_duplicates(), on=["query_id","item_id"], how="outer")
    # label возьмём из первого источника, но он у всех одинаков (label 0/1 для пары)
    lab = runs_oof[rid0][["query_id","item_id"]].merge(
        RUNS[rid0]["oof"][ ["query_id","item_id","label"] ],
        on=["query_id","item_id"], how="left"
    )["label"]
    base["label"] = lab.fillna(0).astype("int8")
    # присоединяем скоры
    for rid in keys:
        base = base.merge(runs_oof[rid][["query_id","item_id","score"]].rename(columns={"score":f"s_{rid}"}),
                          on=["query_id","item_id"], how="left")
    return base

def compute_rrf(df_scores: pd.DataFrame, rids: List[str], rrf_k:int=60):
    # Для RRF считаем ранги по каждой модели отдельно (чем меньше — тем лучше), отсутствию ставим большой ранг
    df = df_scores.copy()
    for rid in rids:
        col = f"s_{rid}"
        # ранги внутри query; если NaN — поставим плохой ранг = max_rank + 1000
        df = df.sort_values(["query_id", col], ascending=[True, False])
        df[f"rk_{rid}"] = df.groupby("query_id")[col].rank(ascending=False, method="first")
    # max rank per query
    max_rk = df.groupby("query_id")[[f"rk_{rid}" for rid in rids]].max().max(axis=1)
    max_rk = max_rk.rename("maxrk")
    df = df.merge(max_rk, left_on="query_id", right_index=True, how="left")
    score = np.zeros(len(df), dtype="float32")
    for rid in rids:
        rk = df[f"rk_{rid}"].values
        rk = np.where(np.isnan(df[f"s_{rid}"].values), df["maxrk"].values + 1000.0, rk)
        score += 1.0/(rrf_k + rk)
    return score

def compute_borda(df_scores: pd.DataFrame, rids: List[str]):
    df = df_scores.copy()
    score = np.zeros(len(df), dtype="float32")
    for rid in rids:
        col = f"s_{rid}"
        df = df.sort_values(["query_id", col], ascending=[True, False])
        rk = df.groupby("query_id")[col].rank(ascending=False, method="first")
        # Borda: (K + 1 - rank)
        K = df.groupby("query_id").size().rename("K")
        df = df.merge(K, left_on="query_id", right_index=True, how="left")
        part = (df["K"] + 1 - rk).values.astype("float32")
        part[np.isnan(df[col].values)] = 0.0
        score += part
        df = df.drop(columns=["K"])
    return score

def compute_combsum(df_scores: pd.DataFrame, rids: List[str]):
    score = np.zeros(len(df_scores), dtype="float32")
    for rid in rids:
        col = f"s_{rid}"
        v = df_scores[col].fillna(0.0).values
        score += v.astype("float32")
    return score

def compute_combmnz(df_scores: pd.DataFrame, rids: List[str]):
    score = compute_combsum(df_scores, rids)
    present = np.zeros(len(df_scores), dtype="int32")
    for rid in rids:
        present += (~df_scores[f"s_{rid}"].isna()).astype("int32").values
    return (score * present).astype("float32")

In [ ]:
btn_simple = W.Button(description="Оценить RRF/Borda/Comb* на OOF", button_style="info", layout=BTN_LAYOUT)
out_simple = W.Output()

EVALS = dict(simple_table=None, union_oof=None, rids=None, per_method_scores=None)

def on_simple(_):
    with out_simple:
        clear_output()
        if not NORMED:
            print("Сначала нормализуй/калибруй OOF.")
            return
        # Union OOF по runs
        rids = list(NORMED.keys())
        runs_oof = {rid: NORMED[rid]["oof"][ ["query_id","item_id","score"] ] for rid in rids}
        union = union_queries_items(runs_oof)
        EVALS["union_oof"] = union
        EVALS["rids"] = rids
        y = union["label"].values
        # Считаем фьюжны
        rows=[]; per_method={}
        if BLEND["use_rrf"]:
            s = compute_rrf(union, rids, BLEND["rrf_k"])
            m = recall_ndcg_at_k(union["query_id"].values, y, s, ks=EVAL["KS"])
            rows.append(dict(method=f"RRF(k={BLEND['rrf_k']})", **m)); per_method["RRF"]=s
        if BLEND["use_borda"]:
            s = compute_borda(union, rids)
            m = recall_ndcg_at_k(union["query_id"].values, y, s, ks=EVAL["KS"])
            rows.append(dict(method="Borda", **m)); per_method["Borda"]=s
        if BLEND["use_combsum"]:
            s = compute_combsum(union, rids)
            m = recall_ndcg_at_k(union["query_id"].values, y, s, ks=EVAL["KS"])
            rows.append(dict(method="CombSUM", **m)); per_method["CombSUM"]=s
        if BLEND["use_combmnz"]:
            s = compute_combmnz(union, rids)
            m = recall_ndcg_at_k(union["query_id"].values, y, s, ks=EVAL["KS"])
            rows.append(dict(method="CombMNZ", **m)); per_method["CombMNZ"]=s
        df = pd.DataFrame(rows).sort_values(f"ndcg@{EVAL['MAIN_K']}", ascending=False) if rows else pd.DataFrame()
        EVALS["simple_table"]=df; EVALS["per_method_scores"]=per_method
        display(df if len(df) else "Нет включённых фьюжнов")

btn_simple.on_click(on_simple)
W.VBox([btn_simple, out_simple])

In [ ]:
btn_weight = W.Button(description="Поиск весов (OOF)", button_style="warning", layout=BTN_LAYOUT)
out_weight = W.Output()

WEIGHTS = dict(best=None, table=None)

def _init_weights(n, nonneg=True, sum_to_one=True):
    w = np.random.rand(n).astype("float32")
    if not nonneg:
        w = (w - 0.5)*2.0
    if sum_to_one and nonneg:
        w = w/ (w.sum()+1e-9)
    return w

def _project_weights(w, nonneg=True, sum_to_one=True):
    v = w.copy()
    if nonneg:
        v = np.maximum(v, 0.0)
    if sum_to_one and nonneg:
        s = v.sum()
        if s>0: v = v/s
        else: v = np.ones_like(v)/len(v)
    return v

def on_weight(_):
    with out_weight:
        clear_output()
        if not EVALS.get("union_oof"):
            print("Сначала посчитай простые фьюжны (они готовят union OOF).")
            return
        union = EVALS["union_oof"]
        rids  = EVALS["rids"]
        y = union["label"].values
        X = np.vstack([union[f"s_{rid}"].fillna(0.0).values.astype("float32") for rid in rids]).T  # [N, M]
        M = X.shape[1]
        best_w=None; best_score=-1; trials=BLEND["weight_trials"]
        mainK = EVAL["MAIN_K"]
        for t in range(trials):
            if t==0:
                w = np.ones(M, dtype="float32")/M
            else:
                # координатное встряхивание: слегка варьируем лучшую точку
                if best_w is not None and np.random.rand()<0.7:
                    w = best_w + 0.1*np.random.randn(M).astype("float32")
                else:
                    w = _init_weights(M, BLEND["weight_nonneg"], BLEND["weight_sum_to_one"])
            w = _project_weights(w, BLEND["weight_nonneg"], BLEND["weight_sum_to_one"])
            s = X.dot(w)
            m = recall_ndcg_at_k(union["query_id"].values, y, s, ks=[mainK])[f"ndcg@{mainK}"]
            if m>best_score:
                best_score=m; best_w=w.copy()
        WEIGHTS["best"] = dict(weights={rid: float(best_w[i]) for i,rid in enumerate(rids)}, ndcg_at_main=float(best_score))
        print("Лучшие веса:", WEIGHTS["best"])

btn_weight.on_click(on_weight)
W.VBox([btn_weight, out_weight])

In [ ]:
btn_meta = W.Button(description="Level-2 (Ridge) на OOF", button_style="success", layout=BTN_LAYOUT)
out_meta = W.Output()

META = dict(oof_scores=None, coefs=None, alpha=None, metrics=None)

def on_meta(_):
    with out_meta:
        clear_output()
        if not EVALS.get("union_oof"):
            print("Сначала подготовь union OOF.")
            return
        if Ridge is None or KFold is None:
            print("sklearn не установлен (Ridge/KFold) — meta недоступен."); return
        union = EVALS["union_oof"]; rids=EVALS["rids"]; y = union["label"].values.astype(int)
        X = np.vstack([union[f"s_{rid}"].fillna(0.0).values.astype("float32") for rid in rids]).T
        q = union["query_id"].values
        uniq_q = np.unique(q)
        kf = KFold(n_splits=BLEND["meta_splits"], shuffle=True, random_state=STATE["RANDOM_SEED"])
        alphas = [1e-4, 1e-3, 1e-2, 1e-1, 1.0]
        best_alpha=None; best_score=-1.0; best_oof=None; best_coefs=None
        for a in alphas:
            oof_pred = np.zeros(len(X), dtype="float32")
            coefs_fold=[]
            for tr_q, va_q in kf.split(uniq_q):
                tr_mask = np.isin(q, uniq_q[tr_q]); va_mask = np.isin(q, uniq_q[va_q])
                model = Ridge(alpha=a, fit_intercept=True, random_state=STATE["RANDOM_SEED"])
                model.fit(X[tr_mask], y[tr_mask])
                oof_pred[va_mask] = model.predict(X[va_mask]).astype("float32")
                coefs_fold.append(model.coef_.copy())
            m = recall_ndcg_at_k(q, y, oof_pred, ks=[EVAL["MAIN_K"]])[f"ndcg@{EVAL['MAIN_K']}"]
            if m>best_score:
                best_score=m; best_alpha=a; best_oof=oof_pred; best_coefs=np.mean(coefs_fold, axis=0)
        META["oof_scores"]=best_oof
        META["coefs"]={rid: float(best_coefs[i]) for i,rid in enumerate(rids)}
        META["alpha"]=float(best_alpha)
        META["metrics"]=recall_ndcg_at_k(q, y, best_oof, ks=EVAL["KS"])
        print("Meta-Ridge:", dict(alpha=META["alpha"], **{k:round(v,4) for k,v in META["metrics"].items()}))
        print("Коэфы:", META["coefs"])

btn_meta.on_click(on_meta)
W.VBox([btn_meta, out_meta])

In [ ]:
btn_compare = W.Button(description="Собрать сравнение и выбрать BEST", button_style="primary", layout=BTN_LAYOUT)
out_compare = W.Output()

BEST = dict(name=None, kind=None, params=None, oof_metrics=None)

def on_compare(_):
    with out_compare:
        clear_output()
        if not EVALS.get("union_oof"):
            print("Нет union OOF.")
            return
        union = EVALS["union_oof"]; y = union["label"].values; q = union["query_id"].values
        rows=[]
        # простые
        if EVALS.get("per_method_scores"):
            for name, s in EVALS["per_method_scores"].items():
                m = recall_ndcg_at_k(q, y, s, ks=EVAL["KS"])
                rows.append(dict(method=name, **m))
        # веса
        if WEIGHTS.get("best"):
            rids = EVALS["rids"]
            w = np.array([WEIGHTS["best"]["weights"][rid] for rid in rids], dtype="float32")
            X = np.vstack([union[f"s_{rid}"].fillna(0.0).values.astype("float32") for rid in rids]).T
            s = X.dot(w)
            m = recall_ndcg_at_k(q, y, s, ks=EVAL["KS"])
            rows.append(dict(method="WeightSearch", **m))
        # meta
        if META.get("oof_scores") is not None:
            m = META["metrics"]
            rows.append(dict(method="Level2_Ridge", **m))
        df = pd.DataFrame(rows).sort_values(f"ndcg@{EVAL['MAIN_K']}", ascending=False) if rows else pd.DataFrame()
        display(df if len(df) else "Нет методов для сравнения")
        if len(df):
            top = df.iloc[0]
            BEST["name"]=str(top["method"])
            BEST["kind"]=BEST["name"]
            # параметры для воспроизводимости
            if BEST["name"].startswith("RRF"):
                BEST["params"]=dict(rrf_k=BLEND["rrf_k"])
            elif BEST["name"]=="WeightSearch":
                BEST["params"]=WEIGHTS["best"]["weights"]
            elif BEST["name"]=="Level2_Ridge":
                BEST["params"]=dict(alpha=META["alpha"], coefs=META["coefs"])
            else:
                BEST["params"]={}
            BEST["oof_metrics"]=top.to_dict()
            print("
→ BEST:", BEST["name"], "| ndcg@{} = {:.5f}".format(EVAL["MAIN_K"], top[f"ndcg@{EVAL['MAIN_K']}"]))

btn_compare.on_click(on_compare)
W.VBox([btn_compare, out_compare])

In [ ]:
btn_slices = W.Button(description="OOF срезы для BEST", button_style="", layout=BTN_LAYOUT)
out_slices = W.Output()

SLICES = dict(result=None)

def on_slices(_):
    with out_slices:
        clear_output()
        if not BEST.get("name"):
            print("Сначала выбери BEST (сводная таблица).")
            return
        union = EVALS["union_oof"]; y = union["label"].values; q = union["query_id"].values
        # получаем scores для BEST
        if BEST["name"] in (EVALS.get("per_method_scores") or {}):
            s = EVALS["per_method_scores"][BEST["name"]]
        elif BEST["name"]=="WeightSearch":
            rids = EVALS["rids"]
            w = np.array([WEIGHTS["best"]["weights"][rid] for rid in rids], dtype="float32")
            X = np.vstack([union[f"s_{rid}"].fillna(0.0).values.astype("float32") for rid in rids]).T
            s = X.dot(w)
        elif BEST["name"]=="Level2_Ridge":
            s = META["oof_scores"]
        else:
            print("Неизвестный BEST."); return

        # Подтягиваем срезы (если есть) из 03_ranker_lab артефактов нельзя — поэтому смотрим флаги в union нет.
        # Попробуем подобрать простые прокси (если в union есть is_cold_item/is_cold_user — скорее нет).
        # Здесь просто выведем базовую метрику + список worst-queries без хита:
        base = recall_ndcg_at_k(q, y, s, ks=[EVAL["MAIN_K"]])
        print("BEST OOF:", base)
        nohit = pd.DataFrame({"q":q, "y":y, "s":s}).sort_values(["q","s"], ascending=[True,False])                 .groupby("q").head(EVAL["MAIN_K"]).groupby("q")["y"].max().reset_index()
        miss = nohit[nohit["y"]==0]["q"].values
        print(f"Queries без хита @ {EVAL['MAIN_K']}: {len(miss)}")
        SLICES["result"]=dict(main=base, nohit_count=int(len(miss)))

btn_slices.on_click(on_slices)
W.VBox([btn_slices, out_slices])

In [ ]:
btn_boot = W.Button(description="Бутстрэп стабильности", button_style="info", layout=BTN_LAYOUT)
out_boot = W.Output()

STAB = dict(ci=None, dist=None)

def _bootstrap_ndcg(q, y, s, B=200, K=20, seed=42):
    rng = np.random.default_rng(seed)
    uq = np.unique(q)
    vals=[]
    for _ in range(B):
        sample_q = rng.choice(uq, size=len(uq), replace=True)
        mask = np.isin(q, sample_q)
        m = recall_ndcg_at_k(q[mask], y[mask], s[mask], ks=[K])[f"ndcg@{K}"]
        vals.append(m)
    vals = np.array(vals, dtype="float64")
    lo, hi = np.percentile(vals, [2.5, 97.5])
    return dict(mean=float(vals.mean()), ci=[float(lo), float(hi)], samples=vals)

def on_boot(_):
    with out_boot:
        clear_output()
        if not BEST.get("name"):
            print("Нет BEST.")
            return
        union = EVALS["union_oof"]; y = union["label"].values; q = union["query_id"].values
        # scores для BEST
        if BEST["name"] in (EVALS.get("per_method_scores") or {}):
            s = EVALS["per_method_scores"][BEST["name"]]
        elif BEST["name"]=="WeightSearch":
            rids = EVALS["rids"]
            w = np.array([WEIGHTS["best"]["weights"][rid] for rid in rids], dtype="float32")
            X = np.vstack([union[f"s_{rid}"].fillna(0.0).values.astype("float32") for rid in rids]).T
            s = X.dot(w)
        elif BEST["name"]=="Level2_Ridge":
            s = META["oof_scores"]
        else:
            print("Неизвестный BEST."); return
        res = _bootstrap_ndcg(q, y, s, B=200, K=EVAL["MAIN_K"], seed=STATE["RANDOM_SEED"])
        STAB.update(ci=res["ci"], dist=res["samples"])
        print(f"Бутстрэп ndcg@{EVAL['MAIN_K']}: mean={res['mean']:.5f}, 95% CI={res['ci']}")

btn_boot.on_click(on_boot)
W.VBox([btn_boot, out_boot])

In [ ]:
btn_infer = W.Button(description="Инференс на TEST (BEST) и сохранение", button_style="danger", layout=BTN_LAYOUT)
out_infer = W.Output()

FILES = dict(ens_dir=None, ranked=None, scores=None)

def _build_test_union() -> pd.DataFrame:
    # Union test по нормализованным runs: вернём (query_id,item_id, s_<rid>...)
    rids = list(NORMED.keys())
    base=None
    for rid in rids:
        df = NORMED[rid]["test"][ ["query_id","item_id","score"] ].rename(columns={"score":f"s_{rid}"})
        base = df if base is None else base.merge(df, on=["query_id","item_id"], how="outer")
    return base

def on_infer(_):
    with out_infer:
        clear_output()
        if not BEST.get("name"):
            print("Выбери BEST.")
            return
        A = art_paths(STATE["DATASET_ID"], STATE["PROFILE_PATH"])
        ens_id = f"{now_tag()}_{BEST['name'].lower()}"
        ens_dir = A["ens"]/ens_id
        ens_dir.mkdir(parents=True, exist_ok=True)

        test_union = _build_test_union()
        rids = list(NORMED.keys())
        # собираем итоговые скоры
        if BEST["name"].startswith("RRF"):
            # для RRF нужны ранги по каждой модели
            tmp = test_union.copy()
            for rid in rids:
                col = f"s_{rid}"
                tmp = tmp.sort_values(["query_id", col], ascending=[True, False])
                tmp[f"rk_{rid}"] = tmp.groupby("query_id")[col].rank(ascending=False, method="first")
            max_rk = tmp.groupby("query_id")[[f"rk_{rid}" for rid in rids]].max().max(axis=1).rename("maxrk")
            tmp = tmp.merge(max_rk, left_on="query_id", right_index=True, how="left")
            s = np.zeros(len(tmp), dtype="float32")
            for rid in rids:
                rk = tmp[f"rk_{rid}"].values
                rk = np.where(np.isnan(tmp[f"s_{rid}"].values), tmp["maxrk"].values + 1000.0, rk)
                s += 1.0/(BLEND["rrf_k"] + rk)
            scores = s
            out = tmp[["query_id","item_id"]].copy()
        elif BEST["name"]=="Borda":
            tmp = test_union.copy()
            s = np.zeros(len(tmp), dtype="float32")
            for rid in rids:
                col = f"s_{rid}"
                tmp = tmp.sort_values(["query_id", col], ascending=[True, False])
                rk = tmp.groupby("query_id")[col].rank(ascending=False, method="first")
                K = tmp.groupby("query_id").size().rename("K")
                tmp = tmp.merge(K, left_on="query_id", right_index=True, how="left")
                part = (tmp["K"] + 1 - rk).values.astype("float32")
                part[np.isnan(tmp[col].values)] = 0.0
                s += part
                tmp = tmp.drop(columns=["K"])
            scores = s
            out = tmp[["query_id","item_id"]].copy()
        elif BEST["name"]=="CombSUM":
            out = test_union[["query_id","item_id"]].copy()
            scores = compute_combsum(test_union, rids)
        elif BEST["name"]=="CombMNZ":
            out = test_union[["query_id","item_id"]].copy()
            scores = compute_combmnz(test_union, rids)
        elif BEST["name"]=="WeightSearch":
            w = np.array([WEIGHTS["best"]["weights"][rid] for rid in rids], dtype="float32")
            X = np.vstack([test_union[f"s_{rid}"].fillna(0.0).values.astype("float32") for rid in rids]).T
            scores = X.dot(w)
            out = test_union[["query_id","item_id"]].copy()
        elif BEST["name"]=="Level2_Ridge":
            coefs = np.array([META["coefs"][rid] for rid in rids], dtype="float64")
            X = np.vstack([test_union[f"s_{rid}"].fillna(0.0).values.astype("float64") for rid in rids]).T
            scores = X.dot(coefs)
            out = test_union[["query_id","item_id"]].copy()
        else:
            print("Неизвестный BEST"); return

        out["score"]=scores.astype("float32")
        out = out.sort_values(["query_id","score"], ascending=[True,False]).groupby("query_id").head(STATE["K_OUT"]).reset_index(drop=True)
        out["rank"] = out.groupby("query_id")["score"].rank(ascending=False, method="first").astype("int32")

        out.to_parquet(ens_dir/"ranked.parquet", index=False)
        test_union.assign(score_blend=scores).to_parquet(ens_dir/"scores_blend.parquet", index=False)

        FILES.update(ens_dir=str(ens_dir), ranked=str(ens_dir/"ranked.parquet"), scores=str(ens_dir/"scores_blend.parquet"))

        print("✓ saved:", ens_dir/"ranked.parquet")

btn_infer.on_click(on_infer)
W.VBox([btn_infer, out_infer])

In [ ]:
btn_save = W.Button(description="Сохранить конфиг/метрики/репорт", button_style="success", layout=BTN_LAYOUT)
out_save = W.Output()

def on_save(_):
    with out_save:
        clear_output()
        if not FILES.get("ens_dir"):
            print("Сначала сделай инференс на test.")
            return
        ens_dir = Path(FILES["ens_dir"])
        # blender_config
        cfg = dict(
            dataset_id=STATE["DATASET_ID"],
            profile=STATE["PROFILE_PATH"],
            runs=list(NORMED.keys()),
            normalize=NORM["method"],
            calibrate=NORM["calibrate"],
            method=BEST["name"],
            params=BEST.get("params", {}),
            K_OUT=STATE["K_OUT"]
        )
        save_json(ens_dir/"blender_config.json", cfg)
        # oof_metrics: соберём сводные по сравнениям, если были
        if out_compare.outputs:
            # На всякий случай пересоберём таблицу из BEST["oof_metrics"] и simple/weight/meta, если есть
            rows=[]
            if EVALS.get("simple_table") is not None and len(EVALS["simple_table"]):
                rows += EVALS["simple_table"].to_dict(orient="records")
            if WEIGHTS.get("best"):
                rows.append(dict(method="WeightSearch", **{k:v for k,v in WEIGHTS["best"].items() if k.startswith("ndcg@")==False}))
            if META.get("metrics"):
                rows.append(dict(method="Level2_Ridge", **META["metrics"]))
            save_json(ens_dir/"oof_metrics.json", dict(rows=rows, best=BEST["oof_metrics"]))
        # slices
        if SLICES.get("result"):
            save_json(ens_dir/"slices.json", SLICES["result"])
        # stability
        if STAB.get("ci") is not None:
            save_json(ens_dir/"stability.json", dict(ci=STAB["ci"]))
        # report
        rep = dict(
            dataset_id=STATE["DATASET_ID"],
            profile=STATE["PROFILE_PATH"],
            ens_id=ens_dir.name,
            best=BEST,
            files=dict(ranked=FILES.get("ranked"), scores=FILES.get("scores")),
            saved_at=datetime.utcnow().isoformat()+"Z"
        )
        save_json(ens_dir/"report.json", rep)
        print("✓ saved config & report →", ens_dir)

btn_save.on_click(on_save)
W.VBox([btn_save, out_save])

In [ ]:
btn_panic = W.Button(description="⚡ PANIC: zscore + RRF + инференс", button_style="danger", layout=BTN_LAYOUT)
out_panic = W.Output()

def on_panic(_):
    with out_panic:
        clear_output()
        # применим zscore_query+none, RRF(k=60)
        NORM["method"]="zscore_query"; NORM["calibrate"]="none"
        BLEND.update(use_rrf=True, rrf_k=60, use_borda=False, use_combsum=False, use_combmnz=False, use_weight_search=False, use_meta_l2=False)
        on_norm(None)
        on_simple(None)
        # выберем RRF как BEST
        if "RRF" in (EVALS.get("per_method_scores") or {}):
            BEST.update(name="RRF", kind="RRF", params=dict(rrf_k=BLEND["rrf_k"]), oof_metrics=EVALS["simple_table"].query("method.str.contains('RRF')", engine="python").iloc[0].to_dict())
            on_infer(None)
            on_save(None)
        else:
            print("PANIC: не удалось посчитать RRF (нет данных).")

btn_panic.on_click(on_panic)
W.VBox([btn_panic, out_panic])

In [ ]:
btn_all = W.Button(description="🚀 Быстрый прогон всего пайплайна", button_style="success", layout=BTN_LAYOUT)
out_all = W.Output()

def on_all(_):
    with out_all:
        clear_output()
        try:
            on_scan(None)
            on_load(None)
            on_norm(None)
            on_simple(None)
            on_weight(None)
            on_meta(None)
            on_compare(None)
            on_slices(None)
            on_boot(None)
            on_infer(None)
            on_save(None)
            print("
✓ Готово.")
        except Exception as e:
            print("ОШИБКА:", e)
            raise

btn_all.on_click(on_all)
W.VBox([btn_all, out_all])

### Памятка
- Нормализация — **per-query** (`zscore/minmax/quantile`) → опц. калибровка по OOF (Platt/Isotonic).
- Фьюжн: RRF/Borda/Comb* быстры и робастны; весовой бленд чаще даёт +0.002…0.01 ndcg@20; Level-2 (Ridge) — когда модели достаточно декоррелированы.
- **Стабильность**: бутстрэп по queries (200 выборок) — смотри CI; если BEST ≈ одиночному ранкеру, просто возьми победителя.

**Стоп-флаги**
- ndcg@20 бленда ≤ лучшего соло (или < +0.001) — не стоит брать бленд.
- Увеличились queries без хита @K — пересмотри union/нормализацию.
- Сильная зависимость от одной модели — можно взять именно её.

Дальше: `05_submit.ipynb` — упаковка сабмита.